<p>26 de febrero de 2024</p>
<p>Universidad Autónoma de Aguascalientes</p>
<p>Departamento: Ciencias de la computación</p>
<p>Carrera: Ingeniería en Computación Inteligente</p>
<p>Maestro: Dr. Francisco Javier Luna Rosas</p>
<p>Curso: Machine learning y deep learning</p>
<p>Alumno: Antonio Muñoz Barrientos</p>
<p>Semestre Enero - junio 2024</p>

# Red neuronal para detectar las firmas

### Librerías

In [93]:
import cv2
import numpy as np
import pandas as pd

## Lectura de la firma

In [94]:
imagen_firma = cv2.imread("Extracted/Firma Extraida 1.jpg", cv2.IMREAD_GRAYSCALE)
imagen_firma.shape

(76, 170)

#### Escalado de la firma
Para mejorar los resultados de los parámetros y tener información con mejor consistencia, vamos a rescalar todas las firmas a una misma resolución de 300x300

In [95]:
imagen_firma = cv2.resize(imagen_firma, (300,300))
imagen_firma.shape

(300, 300)

### Obtener parámetros de la firma


#### Perímetro y área

Se busca encontrar ciertos parámetros generales que nos permitan identificar rasgos simples de una firma, buscaremos al área de la firma y su perímetro. Ya que cada firma tiene es diferente, estos valores deberían cambiar, pero no ser tan diferentes siendo la misma firma.

In [96]:
def obtener_parametros_firma(imagen_firma):

    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(imagen_firma.copy(), cv2.MORPH_OPEN, kernel, iterations=2)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)
    contornos, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if(contornos):
        if (len(contornos) <= 1):
            area_firma = cv2.contourArea(contornos[0])
            perimetro_firma = cv2.arcLength(contornos[0], True)
        else:
            area_firma = cv2.contourArea(contornos[1])
            perimetro_firma = cv2.arcLength(contornos[1], True)
    else:
        area_firma = 0
        perimetro_firma = 0

    return area_firma, perimetro_firma

area, perimetro = obtener_parametros_firma(imagen_firma=imagen_firma)
print("Área de la firma:", area)
print("Perímetro de la firma:", perimetro)

Área de la firma: 89401.0
Perímetro de la firma: 1196.0


#### Elementos estructurales

Para obtener elementos estructurales vamos a utilizar 3 morfologías incluídas en la librería de cv2: MORPH_ELLIPSE, MORPH_RECT, MORPH_CROSS. Vamos a crear un Kerlnel de cada estructura y después vamos a erosionar la imagen con esa estructura, sumaremos los bits prendidos (en 1) que signfica que es donde se encontraron estructuras de dicho elemento en la firma original. La suma es nuestroe elemento a considerar

##### Función para visualizar una imagen cuando sea necesario

In [97]:
def printI(imagen):
    cv2.imshow('Imagen', imagen)
    
    cv2.waitKey(0)

##### Obtener elementos estructurales

In [98]:
def obtener_elementos_estruturales(imagen):
    elementos_estructurantes = []
    for i in range(3, 10, 2):
        elemento = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (i, i))
        elementos_estructurantes.append(elemento)

    for i in range(11, 20, 2):
        elemento = cv2.getStructuringElement(cv2.MORPH_RECT, (i, i))
        elementos_estructurantes.append(elemento)

    for i in range(21, 28, 2):
        elemento = cv2.getStructuringElement(cv2.MORPH_CROSS, (i, i))
        elementos_estructurantes.append(elemento)

    caracteristicas = []
    for elemento in elementos_estructurantes:
        erosion = cv2.erode(imagen, elemento)
        pixeles_encendidos = np.sum(erosion > 0)
        caracteristicas.append(pixeles_encendidos)

    return caracteristicas

caracteristicas = obtener_elementos_estruturales(imagen_firma)
print(caracteristicas)

[77343, 71802, 68519, 65235, 59617, 56343, 53357, 50557, 47956, 57244, 55767, 54411, 53101]


### Creación del dataframe

Solo nos hace falta juntar los elementos para obtener el df correspondiente a la firma actual

<h7>Ejemplo de una entrada del DF</h7>

In [99]:
entrada = {
    'Firma':'5',
    'area':area,
    'perimetro': perimetro,
    'EE1': caracteristicas[0],
    'EE2': caracteristicas[1],
    'EE3': caracteristicas[2],
    'EE4': caracteristicas[3],
    'EE5': caracteristicas[4],
    'EE6': caracteristicas[5],
    'EE7': caracteristicas[6],
    'EE8': caracteristicas[7],
    'EE9': caracteristicas[8],
    'EE10': caracteristicas[9],
    'EE11': caracteristicas[10],
    'EE12': caracteristicas[11],
    'EE12': caracteristicas[12],
}

df = pd.DataFrame(entrada, index=[0])
df

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,5,89401.0,1196.0,77343,71802,68519,65235,59617,56343,53357,50557,47956,57244,55767,53101


## Realizar todo el proceso para todas las firmas

In [100]:
import os
import re

#Diccionario de datos

datos_firmas = {
    'Firma':list(),
    'area':list(),
    'perimetro': list(),
    'EE1': list(),
    'EE2': list(),
    'EE3': list(),
    'EE4': list(),
    'EE5': list(),
    'EE6': list(),
    'EE7': list(),
    'EE8': list(),
    'EE9': list(),
    'EE10': list(),
    'EE11': list(),
    'EE12': list(),
    'EE12': list(),
}

# Leer correctamente la carpeta
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

ruta = "Extracted/"

firmas = sorted_alphanumeric(os.listdir(ruta))

for i,firma in enumerate(firmas):
    imagen_firma = cv2.imread(ruta+firma, cv2.IMREAD_GRAYSCALE)
    imagen_firma = cv2.resize(imagen_firma, (300,300))
    
    area, perimetro = obtener_parametros_firma(imagen_firma=imagen_firma.copy())
    caracteristicas = obtener_elementos_estruturales(imagen_firma.copy())
    datos_firmas['Firma'].append(i+1)
    datos_firmas['area'].append(area)
    datos_firmas['perimetro'].append(perimetro)
    for j,element in enumerate(list(datos_firmas.keys())[3:]):
        datos_firmas[element].append(caracteristicas[j])

dataframe = pd.DataFrame(datos_firmas, index=range(len(datos_firmas)))
dataframe = dataframe.round(4)
dataframe

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,1,89401.0,1196.0000,77343,71802,68519,65235,59617,56343,53357,50557,47956,57244,55767,54411
1,2,89401.0,1196.0000,75624,69832,66454,63108,57309,54050,50913,47893,45031,54335,52598,50933
2,3,85728.5,1456.4680,78115,72815,69442,65994,59750,56359,53209,50421,47884,57266,55670,54217
3,4,89401.0,1196.0000,76647,70481,66758,63051,56611,53175,50050,47149,44470,54444,52771,51155
4,5,87465.5,1578.4092,78157,72959,69448,65947,59892,56600,53631,50915,48376,57703,56166,54728
5,6,87994.0,1431.2548,77357,72169,68963,65796,60388,57441,54797,52352,50171,56762,55161,53678
6,7,89378.0,1202.8284,74758,68345,64566,60953,54967,51435,48150,45105,42418,50945,48969,47200
7,8,87638.0,1566.3675,78412,73394,70123,66827,61416,58358,55546,52853,50271,59095,57652,56265
8,9,89339.0,1207.6569,80841,74080,69921,66284,60573,57371,54510,51972,49819,59659,58093,56580
9,10,87800.5,1486.4680,76254,70650,67319,64050,58500,55405,52615,49988,47574,55704,54197,52782


## Crear nuevas entradas
Para mejorar la precisión de nuestra red neuronal, crearemos nuevas entradas calculadas a partir de las anteriores, utilizaremos la media y la desviación estándar de cada entrada para crear nuevas entradas. El objetivo es tener un dataset de nuevas entradas.

In [101]:
import random
datos_firmas = {
    'Firma':list(),
    'area':list(),
    'perimetro': list(),
    'EE1': list(),
    'EE2': list(),
    'EE3': list(),
    'EE4': list(),
    'EE5': list(),
    'EE6': list(),
    'EE7': list(),
    'EE8': list(),
    'EE9': list(),
    'EE10': list(),
    'EE11': list(),
    'EE12': list(),
    'EE12': list(),
}

for i in range(16,61):
    datos_firmas["Firma"].append(i)
    area_mean = dataframe['area'].mean()
    area_standar = dataframe['area'].std()
    perimetro_mean = dataframe['perimetro'].mean()
    perimetro_standar = dataframe['perimetro'].std()
    random_value = random.randint(1,3)
    if(random_value < 2):
        datos_firmas['area'].append(area_mean - area_standar)
        datos_firmas['perimetro'].append(perimetro_mean - perimetro_standar)
        
        for _,element in enumerate(list(datos_firmas.keys())[3:]):
            EE_mean = dataframe[element].mean()
            EE_std = dataframe[element].std()
            datos_firmas[element].append(EE_mean - EE_std)
    elif(random_value > 2):
        datos_firmas['area'].append(area_mean + area_standar)
        datos_firmas['perimetro'].append(perimetro_mean + perimetro_standar)
        for _,element in enumerate(list(datos_firmas.keys())[3:]):
            EE_mean = dataframe[element].mean()
            EE_std = dataframe[element].std()
            datos_firmas[element].append(EE_mean + EE_std)
    else:
        datos_firmas['area'].append(area_mean)
        datos_firmas['perimetro'].append(perimetro_mean)
        for _,element in enumerate(list(datos_firmas.keys())[3:]):
            EE_mean = dataframe[element].mean()
            datos_firmas[element].append(EE_mean)
            
    dataframe_generado = pd.DataFrame(datos_firmas, index=[0])
    dataframe = pd.concat([dataframe, dataframe_generado], ignore_index=True)
    datos_firmas = {
    'Firma':list(),
    'area':list(),
    'perimetro': list(),
    'EE1': list(),
    'EE2': list(),
    'EE3': list(),
    'EE4': list(),
    'EE5': list(),
    'EE6': list(),
    'EE7': list(),
    'EE8': list(),
    'EE9': list(),
    'EE10': list(),
    'EE11': list(),
    'EE12': list(),
    'EE12': list(),
    }

dataframe

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,1,89401.000000,1196.000000,77343.000000,71802.000000,68519.000000,65235.000000,59617.000000,56343.000000,53357.000000,50557.000000,47956.000000,57244.000000,55767.000000,54411.000000
1,2,89401.000000,1196.000000,75624.000000,69832.000000,66454.000000,63108.000000,57309.000000,54050.000000,50913.000000,47893.000000,45031.000000,54335.000000,52598.000000,50933.000000
2,3,85728.500000,1456.468000,78115.000000,72815.000000,69442.000000,65994.000000,59750.000000,56359.000000,53209.000000,50421.000000,47884.000000,57266.000000,55670.000000,54217.000000
3,4,89401.000000,1196.000000,76647.000000,70481.000000,66758.000000,63051.000000,56611.000000,53175.000000,50050.000000,47149.000000,44470.000000,54444.000000,52771.000000,51155.000000
4,5,87465.500000,1578.409200,78157.000000,72959.000000,69448.000000,65947.000000,59892.000000,56600.000000,53631.000000,50915.000000,48376.000000,57703.000000,56166.000000,54728.000000
5,6,87994.000000,1431.254800,77357.000000,72169.000000,68963.000000,65796.000000,60388.000000,57441.000000,54797.000000,52352.000000,50171.000000,56762.000000,55161.000000,53678.000000
6,7,89378.000000,1202.828400,74758.000000,68345.000000,64566.000000,60953.000000,54967.000000,51435.000000,48150.000000,45105.000000,42418.000000,50945.000000,48969.000000,47200.000000
7,8,87638.000000,1566.367500,78412.000000,73394.000000,70123.000000,66827.000000,61416.000000,58358.000000,55546.000000,52853.000000,50271.000000,59095.000000,57652.000000,56265.000000
8,9,89339.000000,1207.656900,80841.000000,74080.000000,69921.000000,66284.000000,60573.000000,57371.000000,54510.000000,51972.000000,49819.000000,59659.000000,58093.000000,56580.000000
9,10,87800.500000,1486.468000,76254.000000,70650.000000,67319.000000,64050.000000,58500.000000,55405.000000,52615.000000,49988.000000,47574.000000,55704.000000,54197.000000,52782.000000


### Juntamos los DataFrame e indicamos con una variable booleana que si es nuestra firma

In [102]:
data_frame_correcto = dataframe.round(2)
data_frame_correcto["Valido"] = 1
# Ejemplo de 10 firmas
data_frame_correcto.sample(10)

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12,Valido
42,43,88710.90,1380.48,77662.16,72179.49,68772.05,65432.03,59758.50,56589.44,53677.83,50998.32,48534.05,57069.31,55506.50,54049.24,1
53,54,89693.51,1524.71,79288.82,74095.93,70822.32,67622.75,62148.99,59130.28,56364.08,53817.03,51463.62,59920.30,58483.86,57122.31,1
11,12,89401.00,1196.00,73546.00,67554.00,63884.00,60259.00,54280.00,50846.00,47669.00,44751.00,42062.00,50148.00,48345.00,46745.00,1
19,20,89621.91,1514.20,79170.29,73956.28,70672.92,67463.12,61974.80,58945.13,56168.33,53611.63,51250.15,59712.55,58266.90,56898.38,1
57,58,89701.68,1525.90,79302.33,74111.85,70839.35,67640.95,62168.85,59151.38,56386.39,53840.44,51487.96,59943.98,58508.59,57147.84,1
52,53,88823.01,1396.94,77847.75,72398.15,69005.97,65681.98,60031.24,56879.33,53984.32,51319.92,48868.29,57394.59,55846.20,54399.86,1
37,38,89629.91,1515.37,79183.52,73971.88,70689.60,67480.94,61994.25,58965.81,56190.19,53634.57,51273.99,59735.75,58291.13,56923.39,1
43,44,89620.02,1513.92,79167.15,73952.58,70668.96,67458.89,61970.19,58940.23,56163.15,53606.19,51244.50,59707.05,58261.15,56892.45,1
38,39,88709.50,1380.28,77659.85,72176.77,68769.14,65428.92,59755.10,56585.83,53674.02,50994.32,48529.88,57065.26,55502.27,54044.87,1
22,23,89699.37,1525.57,79298.52,74107.35,70834.54,67635.81,62163.25,59145.43,56380.09,53833.83,51481.09,59937.30,58501.61,57140.64,1


### Debemos crear los datos que no son correctos

Lo haremos con datos random para cada columna y el campo valido igual a 0

In [103]:
datos_firmas = {
    'Firma':list(),
    'Valido': list(),
    'area':list(),
    'perimetro': list(),
    'EE1': list(),
    'EE2': list(),
    'EE3': list(),
    'EE4': list(),
    'EE5': list(),
    'EE6': list(),
    'EE7': list(),
    'EE8': list(),
    'EE9': list(),
    'EE10': list(),
    'EE11': list(),
    'EE12': list(),
    'EE12': list(),
}

for i in range(61,120):
    datos_firmas['Firma'].append(i)
    datos_firmas['Valido'].append(0)
    for _,element in enumerate(list(datos_firmas.keys())[2:]):
            datos_firmas[element].append(random.randint(100,10000))

dataframe_incorrecto = pd.DataFrame(datos_firmas, index=range(59))
dataframe_incorrecto

,Firma,Valido,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,61,0,6825,7563,2684,8555,777,1848,9583,4115,2195,4367,4557,7353,1952,4899
1,62,0,6198,9162,1105,8756,7398,9893,5148,7641,8305,1384,601,3995,3846,9355
2,63,0,8373,4145,8809,3940,6620,449,7021,4838,1347,8380,8002,4733,8685,8253
3,64,0,8356,9810,709,6017,179,9752,6408,3801,5740,4330,7924,8473,3104,347
4,65,0,2350,529,194,1383,8877,7791,1091,5914,1701,9907,502,378,8007,2681
5,66,0,2443,9029,1469,2651,7533,3848,4568,1555,8910,7592,9376,5404,1152,1721
6,67,0,8043,8644,8757,1550,7327,6227,3470,4671,852,8411,6698,4523,4088,3850
7,68,0,3088,4651,4242,957,6272,5363,8114,8830,8888,9931,856,2094,8596,3054
8,69,0,4681,6895,8385,7505,9532,4634,2209,1852,6790,4085,4173,2671,3935,5363
9,70,0,4094,8770,5511,8878,4283,6244,236,2231,1658,8337,396,9656,7119,7330


### Juntamos los dataframes

In [104]:
data_frame_completo = pd.concat([data_frame_correcto, dataframe_incorrecto], ignore_index=True)
data_frame_completo = data_frame_completo.round(2)
data_frame_completo.sample(20)

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12,Valido
14,15,89401.00,1196.00,78813.00,74225.00,71067.00,67909.00,62358.00,59279.00,56435.00,53898.00,51506.00,60377.00,59088.00,57872.00,1
47,48,88770.20,1389.19,77760.34,72295.16,68895.80,65564.26,59902.78,56742.79,53839.96,51168.45,48710.86,57241.38,55686.20,54234.72,1
19,20,89621.91,1514.20,79170.29,73956.28,70672.92,67463.12,61974.80,58945.13,56168.33,53611.63,51250.15,59712.55,58266.90,56898.38,1
36,37,89604.71,1511.67,79141.80,73922.72,70637.01,67424.76,61932.94,58900.64,56121.29,53562.27,51198.85,59662.63,58214.76,56844.57,1
6,7,89378.00,1202.83,74758.00,68345.00,64566.00,60953.00,54967.00,51435.00,48150.00,45105.00,42418.00,50945.00,48969.00,47200.00,1
48,49,89658.66,1519.59,79231.13,74027.96,70749.60,67545.06,62064.21,59040.16,56268.80,53717.06,51359.72,59819.18,58378.26,57013.32,1
90,91,8395.00,4233.00,7112.00,6871.00,1500.00,6325.00,2241.00,9137.00,6641.00,6316.00,8077.00,3522.00,1351.00,4163.00,0
12,13,87709.50,1537.92,79424.00,75028.00,72210.00,69380.00,64361.00,61579.00,58977.00,56529.00,54216.00,61516.00,60091.00,58748.00,1
74,75,2664.00,4141.00,8584.00,6538.00,4329.00,5363.00,1664.00,3162.00,9090.00,5452.00,893.00,6895.00,4182.00,8454.00,0
115,116,1548.00,6132.00,4886.00,3565.00,6101.00,4459.00,2820.00,2152.00,6885.00,1064.00,6756.00,4686.00,6462.00,2759.00,0


In [105]:
### Exportamos el dataFrame
data_frame_completo.to_csv('DataFrame_Firmas.csv', index=False)

# Entrenamiento del modelo

#### Librerías

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

#### Separamos las variables predictoras (X) y la variable objetivo (y) 

In [107]:
X = data_frame_completo.iloc[:,1:15]
X

,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,89401.0,1196.00,77343.0,71802.0,68519.0,65235.0,59617.0,56343.0,53357.0,50557.0,47956.0,57244.0,55767.0,54411.0
1,89401.0,1196.00,75624.0,69832.0,66454.0,63108.0,57309.0,54050.0,50913.0,47893.0,45031.0,54335.0,52598.0,50933.0
2,85728.5,1456.47,78115.0,72815.0,69442.0,65994.0,59750.0,56359.0,53209.0,50421.0,47884.0,57266.0,55670.0,54217.0
3,89401.0,1196.00,76647.0,70481.0,66758.0,63051.0,56611.0,53175.0,50050.0,47149.0,44470.0,54444.0,52771.0,51155.0
4,87465.5,1578.41,78157.0,72959.0,69448.0,65947.0,59892.0,56600.0,53631.0,50915.0,48376.0,57703.0,56166.0,54728.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2629.0,1391.00,7869.0,9347.0,2365.0,6105.0,373.0,2842.0,262.0,8278.0,1575.0,2136.0,2304.0,6857.0
115,1548.0,6132.00,4886.0,3565.0,6101.0,4459.0,2820.0,2152.0,6885.0,1064.0,6756.0,4686.0,6462.0,2759.0
116,9417.0,3846.00,3602.0,6706.0,7124.0,4165.0,4803.0,3822.0,7890.0,9422.0,7792.0,3518.0,4069.0,8207.0
117,7777.0,616.00,1117.0,3406.0,4106.0,3132.0,6006.0,8545.0,6486.0,3568.0,2751.0,472.0,5045.0,9183.0


## Normalizamos los datos 

In [108]:
data_frame_completo.iloc[:,1:15] = StandardScaler().fit_transform(data_frame_completo.iloc[:,1:15])
data_frame_completo.head()

,Firma,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12,Valido
0,1,1.003546,-0.734097,0.974590,0.970064,0.971303,0.971403,0.969712,0.963695,0.956798,0.948259,0.939229,0.977690,0.979131,0.982263,1
1,2,1.003546,-0.734097,0.927471,0.911832,0.906870,0.901337,0.886193,0.876078,0.858256,0.833133,0.808463,0.867829,0.856262,0.841236,1
2,3,0.916177,-0.644335,0.995751,1.000008,1.000103,0.996405,0.974524,0.964306,0.950831,0.942381,0.936010,0.978520,0.975370,0.974397,1
3,4,1.003546,-0.734097,0.955512,0.931016,0.916355,0.899459,0.860935,0.842643,0.823460,0.800981,0.783383,0.871945,0.862970,0.850238,1
4,5,0.957500,-0.602313,0.996903,1.004265,1.000290,0.994857,0.979663,0.973515,0.967846,0.963730,0.958005,0.995024,0.994601,0.995117,1


#### Re asignamos X

In [109]:
X = data_frame_completo.iloc[:,1:15]
X

,area,perimetro,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12
0,1.003546,-0.734097,0.974590,0.970064,0.971303,0.971403,0.969712,0.963695,0.956798,0.948259,0.939229,0.977690,0.979131,0.982263
1,1.003546,-0.734097,0.927471,0.911832,0.906870,0.901337,0.886193,0.876078,0.858256,0.833133,0.808463,0.867829,0.856262,0.841236
2,0.916177,-0.644335,0.995751,1.000008,1.000103,0.996405,0.974524,0.964306,0.950831,0.942381,0.936010,0.978520,0.975370,0.974397
3,1.003546,-0.734097,0.955512,0.931016,0.916355,0.899459,0.860935,0.842643,0.823460,0.800981,0.783383,0.871945,0.862970,0.850238
4,0.957500,-0.602313,0.996903,1.004265,1.000290,0.994857,0.979663,0.973515,0.967846,0.963730,0.958005,0.995024,0.994601,0.995117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,-1.060773,-0.666897,-0.929753,-0.876067,-1.092867,-0.976414,-1.174129,-1.080618,-1.183993,-0.878844,-1.134289,-1.103514,-1.093745,-0.945975
115,-1.086490,0.966912,-1.011519,-1.046980,-0.976294,-1.030635,-1.085580,-1.106984,-0.916954,-1.190600,-0.902666,-1.007211,-0.932530,-1.112142
116,-0.899285,0.179127,-1.046715,-0.954134,-0.944374,-1.040320,-1.013822,-1.043172,-0.876432,-0.829406,-0.856351,-1.051321,-1.025312,-0.891234
117,-0.938301,-0.933972,-1.114831,-1.051680,-1.038543,-1.074348,-0.970290,-0.862702,-0.933041,-1.082388,-1.081715,-1.166356,-0.987470,-0.851659


#### Obtenemos variable objetivo

In [110]:
y = data_frame_completo.iloc[:, -1]
y

0      1
1      1
2      1
3      1
4      1
      ..
114    0
115    0
116    0
117    0
118    0
Name: Valido, Length: 119, dtype: int64

## Separamos el dataset en entramiento 70% y prueba 30%

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=13)

### Inicializamos la red neuornal

In [112]:
instancia_red = MLPClassifier(hidden_layer_sizes=(101,))
print(instancia_red)

MLPClassifier(hidden_layer_sizes=(101,))


In [113]:
instancia_red.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(101,))

### Función para calcular que tan bueno es el modelo

In [114]:
def indices_general(MC, nombres=None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    precision_categorica = pd.DataFrame(MC.diagonal()/np.sum(MC,axis=1)).T
    if nombres != None:
        precision_categorica.columns = nombres
    return {
        "Matriz de confusion": MC,
        "Precision global": precision_global,
        "Error global": error_global,
        "Precision por categoria": precision_categorica
    }

#### Índice de precisión del modelo

In [115]:
prediccion = instancia_red.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de confusion:
[[18  0]
 [ 0 18]]

Precision global:
1.0

Error global:
0.0

Precision por categoria:
     0    1
0  1.0  1.0


### 

# Referencias
- OpenCV Resize image using cv2.resize(): https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/#gsc.tab=0
- Morphological Transformations: https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html
- OpenCV Morphological Operations: https://pyimagesearch.com/2021/04/28/opencv-morphological-operations/
- sklearn.neural_network.MLPClassifier: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
